In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
!pip install gurobipy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 17.4 MB/s 


In [3]:

import os
import sys
# path = os.path.dirname(os.getcwd())
# sys.path.insert(0, path)

sys.path.append('/content/gdrive/MyDrive/SVR_Codes/SVR_CG')
from Codes.model import *
from Codes.utils import *

import pickle
sys.path.append('/content/gdrive/MyDrive/SVR_Codes/DatasetsResult')
RESULT_DIR = "/content/gdrive/MyDrive/SVR_Codes/DatasetsResult"

sys.path.append('/content/gdrive/MyDrive/SVR_Codes/Datasets')
DATA_DIR = "/content/gdrive/MyDrive/SVR_Codes/Datasets"

sys.path.append('/content/gdrive/MyDrive/SVR_Codes/DatasetsInfo')
DATAINFO_DIR = "/content/gdrive/MyDrive/SVR_Codes/DatasetsInfo"

import warnings
warnings.filterwarnings("ignore")



In [4]:
datasets_list = ['Whitewine', 'Ceosalaries', 'Redwine', 'Airfoil','Housing','Powerplant','Autompg','Abalone','Iris','Protein'] #'Zoo',


In [ ]:

def extractData_Info(datasets_list ):

    df_data = pd.DataFrame(np.c_[datasets_list, np.zeros((len(datasets_list),3))] ,columns=(['Dataset', 'n','d' ,'k']))


    for i, data in enumerate(datasets_list):

        df_data.iloc[i,0] = datasets_list[i]

        with open(os.path.join(DATAINFO_DIR, str(data)+'.pkl'), 'rb') as inp:
            n = pickle.load(inp)
            d = pickle.load(inp)
            k = pickle.load(inp)

        df_data.loc[i,'n'] = n
        df_data.loc[i,'d'] = d
        df_data.loc[i,'k'] = k

        # df_data.loc[i,'Data source'] = str('\href{'+data_links[i]+'}{'+data_repo[i]+'}')
    
    return df_data


In [ ]:
df_data =  extractData_Info(datasets_list )
df_data = df_data.sort_values(by=['n'])
print(df_data.to_latex(index=False))


\begin{tabular}{llll}
\toprule
    Dataset &     n &  d & k \\
\midrule
       Iris &   150 &  4 & 3 \\
    Autompg &   392 &  7 & 3 \\
Ceosalaries &   500 &  1 & 6 \\
    Housing &   506 & 13 & 2 \\
    Airfoil &  1503 &  5 & 4 \\
    Redwine &  1599 & 11 & 3 \\
    Abalone &  4177 &  7 & 3 \\
  Whitewine &  4898 & 11 & 3 \\
 Powerplant &  9568 &  4 & 5 \\
    Protein & 45730 &  9 & 2 \\
\bottomrule
\end{tabular}



In [5]:
import time

def extractData_ci(datasets_list ):

    cols = ['km-lr', 'k-plane', 'EM-greedy', 'opt-cg', 'opt-Milp-10','km-svr','opt-MILP','km-et']
    # model = ['kmlr', 'kp', 'greedy', 'optcg', 'optmilp']
    allcols = ['data', 'n','d', 'k', 'km-lr', 'k-plane', 'EM-greedy', 'opt-cg', 'opt-Milp-10','km-svr','opt-MILP','km-et']

    # cols = ['kc-Gon', 'kc-GonAvg', 'kc-HS', 'kmed', 'kmed++', 'Opt-Cg']
    # model = ['kcgon', 'kcgonavg', 'kchs', 'km', 'kmplus', 'mmopt']
    # allcols = ['data', 'n','d', 'kc-Gon', 'kc-GonAvg', 'kc-HS', 'kmed', 'kmed++', 'Opt-Cg']

   
    df_epsVal = pd.DataFrame(np.c_[datasets_list, np.zeros((len(datasets_list),len(cols)+3))] ,columns=allcols)
    df_time = pd.DataFrame(np.c_[datasets_list, np.zeros((len(datasets_list),len(cols)+3))] ,columns=allcols)


    # optgap = []

    for i, data in enumerate(datasets_list):


        with open(os.path.join(DATAINFO_DIR, str(data)+'.pkl'), 'rb') as inp:
            n = pickle.load(inp)
            d = pickle.load(inp)
            k = pickle.load(inp)

 
        df_time.loc[i,'n'] = n
        df_time.loc[i,'d'] = d
        df_time.loc[i,'k'] = k

        df_epsVal.loc[i,'n'] = n
        df_epsVal.loc[i,'d'] = d
        df_epsVal.loc[i,'k'] = k

        time_result = np.zeros((len(cols),1))
        eps_result = np.zeros((len(cols),1))


        with open(os.path.join(RESULT_DIR, str(data)+'.pkl'), 'rb') as inp:



            weights_list_greedy=pickle.load(inp)
            bias_list_greedy=pickle.load(inp)
            time_result_greedy=pickle.load(inp)
            eps_result_greedy=pickle.load(inp)

            weights_list_kmlr=pickle.load(inp)
            bias_list_kmlr=pickle.load(inp)
            time_result_kmlr=pickle.load(inp)
            eps_result_kmlr=pickle.load(inp)
            
            weights_list_kp=pickle.load(inp)
            bias_list_kp=pickle.load(inp)
            time_result_kp=pickle.load(inp)
            eps_result_kp=pickle.load(inp)

            weights_optcg=pickle.load(inp)
            bias_optcg=pickle.load(inp)
            time_result[3]=pickle.load(inp)
            optgap_optcg=pickle.load(inp)
            eps_result[3]=pickle.load(inp)

            weights_optmilp=pickle.load(inp)
            bias_optmilp=pickle.load(inp)
            time_result[4]=pickle.load(inp)
            optgap_optmilp=pickle.load(inp)
            eps_result[4]=pickle.load(inp)



        time_result[2] = time_result_greedy
        time_result[0] = time_result_kmlr
        time_result[1] = time_result_kp

        eps_result[2] = eps_result_greedy[0]
        eps_result[0] = eps_result_kmlr[0]
        eps_result[1] = eps_result_kp[0]


        with open(os.path.join(RESULT_DIR, str(data)+'2.pkl'), 'rb') as inp:

            weights_list_kmsvr=pickle.load(inp)
            bias_list_kmsvr=pickle.load(inp)
            time_result_kmsvr=pickle.load(inp)
            eps_result_kmsvr=pickle.load(inp)

            weights_optmilp=pickle.load(inp)
            bias_optmilp=pickle.load(inp)
            time_result[6]=pickle.load(inp)
            optgap_optmilp=pickle.load(inp)
            eps_result[6]=pickle.load(inp)



        time_result[5] = time_result_kmsvr
        eps_result[5] = eps_result_kmsvr[0]

        with open(os.path.join(RESULT_DIR, str(data)+'3.pkl'), 'rb') as inp:

            weights_list_kmet=pickle.load(inp)
            bias_list_kmet=pickle.load(inp)
            time_result_kmet=pickle.load(inp)
            eps_result_kmet=pickle.load(inp)

        time_result[7] = time_result_kmet
        eps_result[7] = eps_result_kmet[0]

        # print(eps_result)

        for j,co in enumerate(cols):
            # eps = epsVal[j]
            # ari_mod = ari[j]
            # pur = purity[j]
            
            # df_epsVal.loc[i, cols[j]] = '{:.3f}'.format(int(np.around(eps, 3)))
  
            df_epsVal.loc[i, cols[j]] = float('{:.2f}'.format(float(np.around(eps_result[j], 3))))
            df_time.loc[i, cols[j]] = '{:.2f}'.format(float(np.around(time_result[j], 3)))


    return df_epsVal, df_time


In [6]:
df_epsVal, df_time =  extractData_ci(datasets_list )


In [7]:

df_epsVal = df_epsVal.sort_values(by=['n'])
df_time = df_time.sort_values(by=['n'])

In [8]:
df_time

,data,n,d,k,km-lr,k-plane,EM-greedy,opt-cg,opt-Milp-10,km-svr,opt-MILP,km-et
8,Iris,150,4,3,0.06,1.42,0.49,841.16,600.24,0.05,840.24,0.17
6,Autompg,392,7,3,0.06,2.02,0.57,1561.32,600.24,0.32,1561.57,0.23
1,Ceosalaries,500,1,6,0.08,3.57,1.76,240.22,600.28,0.09,240.50,0.29
4,Housing,506,13,2,0.05,1.05,0.61,2642.22,600.25,0.06,2642.45,0.21
3,Airfoil,1503,5,4,0.10,4.92,1.61,3363.41,600.27,0.17,3363.74,0.61
2,Redwine,1599,11,3,1.38,6.15,2.40,2642.76,600.40,1.37,2643.09,1.90
7,Abalone,4177,7,3,1.60,14.73,4.98,2283.63,600.52,1.35,2284.07,2.32
0,Whitewine,4898,11,3,1.50,17.23,6.88,6005.38,600.54,1.74,6005.83,2.54
5,Powerplant,9568,4,5,1.96,31.58,8.32,1201.30,600.96,1.54,1202.01,3.14
9,Protein,45730,9,2,1.68,127.47,23.78,3003.83,602.73,1.80,3006.13,8.74


In [9]:
df_epsVal

,data,n,d,k,km-lr,k-plane,EM-greedy,opt-cg,opt-Milp-10,km-svr,opt-MILP,km-et
8,Iris,150,4,3,0.8,0.48,0.32,0.22,0.23,0.94,0.24,0.63
6,Autompg,392,7,3,1.5,1.0,0.59,0.38,0.42,1.38,0.42,1.06
1,Ceosalaries,500,1,6,10.64,2.5,0.7,0.48,0.62,5.69,0.64,5.68
4,Housing,506,13,2,2.34,1.76,0.99,0.67,0.75,2.02,0.78,1.27
3,Airfoil,1503,5,4,2.77,1.21,0.86,0.59,0.79,2.3,0.77,2.2
2,Redwine,1599,11,3,3.04,2.34,1.55,0.62,1.5,2.81,1.13,2.24
7,Abalone,4177,7,3,4.56,2.82,1.75,0.88,2.07,3.57,1.75,2.83
0,Whitewine,4898,11,3,3.98,3.05,1.86,1.03,2.55,3.39,1.82,2.98
5,Powerplant,9568,4,5,2.71,1.76,0.53,0.3,1.56,1.99,1.45,1.62
9,Protein,45730,9,2,3.94,2.16,1.67,0.82,1.72,1.72,1.72,1.72


In [10]:
cols =['data', 'n','d', 'k', 'km-lr', 'k-plane', 'km-svr', 'km-et', 'EM-greedy', 'opt-MILP','opt-cg']
df = df_epsVal.loc[:,cols]
df = df.set_index('data').T.reset_index()


In [11]:
print(df)

data      index  Iris Autompg Ceosalaries Housing Airfoil Redwine Abalone  \
0             n   150     392         500     506    1503    1599    4177   
1             d     4       7           1      13       5      11       7   
2             k     3       3           6       2       4       3       3   
3         km-lr   0.8     1.5       10.64    2.34    2.77    3.04    4.56   
4       k-plane  0.48     1.0         2.5    1.76    1.21    2.34    2.82   
5        km-svr  0.94    1.38        5.69    2.02     2.3    2.81    3.57   
6         km-et  0.63    1.06        5.68    1.27     2.2    2.24    2.83   
7     EM-greedy  0.32    0.59         0.7    0.99    0.86    1.55    1.75   
8      opt-MILP  0.24    0.42        0.64    0.78    0.77    1.13    1.75   
9        opt-cg  0.22    0.38        0.48    0.67    0.59    0.62    0.88   

data Whitewine Powerplant Protein  
0         4898       9568   45730  
1           11          4       9  
2            3          5       2  
3       

In [12]:
print(df.to_latex(index=False))


\begin{tabular}{lllllllllll}
\toprule
    index &  Iris & Autompg & Ceosalaries & Housing & Airfoil & Redwine & Abalone & Whitewine & Powerplant & Protein \\
\midrule
        n &   150 &     392 &         500 &     506 &    1503 &    1599 &    4177 &      4898 &       9568 &   45730 \\
        d &     4 &       7 &           1 &      13 &       5 &      11 &       7 &        11 &          4 &       9 \\
        k &     3 &       3 &           6 &       2 &       4 &       3 &       3 &         3 &          5 &       2 \\
    km-lr &   0.8 &     1.5 &       10.64 &    2.34 &    2.77 &    3.04 &    4.56 &      3.98 &       2.71 &    3.94 \\
  k-plane &  0.48 &     1.0 &         2.5 &    1.76 &    1.21 &    2.34 &    2.82 &      3.05 &       1.76 &    2.16 \\
   km-svr &  0.94 &    1.38 &        5.69 &    2.02 &     2.3 &    2.81 &    3.57 &      3.39 &       1.99 &    1.72 \\
    km-et &  0.63 &    1.06 &        5.68 &    1.27 &     2.2 &    2.24 &    2.83 &      2.98 &       1.62 &    1

In [ ]:
print(df_time.to_latex(index=False))

\begin{tabular}{lllllllll}
\toprule
       data &    n &  d & k & km-lr & k-plane & EM-greedy &  opt-cg & Opt-Milp \\
\midrule
       Iris &  150 &  4 & 3 &  0.23 &    0.87 &      0.47 & 1321.40 &   600.26 \\
    Autompg &  392 &  7 & 3 &  0.06 &    2.02 &      0.57 & 1561.32 &   600.24 \\
Ceosalaries &  500 &  1 & 6 &  0.08 &    3.57 &      1.76 &  240.22 &   600.28 \\
    Housing &  506 & 13 & 2 &  0.05 &    1.05 &      0.61 & 2642.22 &   600.25 \\
    Airfoil & 1503 &  5 & 4 &  0.10 &    4.92 &      1.61 & 3363.41 &   600.27 \\
    Redwine & 1599 & 11 & 3 &  1.38 &    6.15 &      2.40 & 2642.76 &   600.40 \\
    Abalone & 4177 &  7 & 3 &  1.66 &    5.18 &      2.39 & 1442.09 &   600.35 \\
  Whitewine & 4898 & 11 & 3 &  1.50 &   17.23 &      6.88 & 6005.38 &   600.54 \\
 Powerplant & 9568 &  4 & 5 &  1.96 &   31.58 &      8.32 & 1201.30 &   600.96 \\
\bottomrule
\end{tabular}



In [ ]:
print(df_epsVal.to_latex(index=False))


\begin{tabular}{lllllllll}
\toprule
       data &    n &  d & k &  km-lr & k-plane & EM-greedy & opt-cg & Opt-Milp \\
\midrule
       Iris &  150 &  4 & 3 &   0.59 &    0.37 &      0.26 &   0.17 &     0.18 \\
    Autompg &  392 &  7 & 3 &    1.5 &     1.0 &      0.59 &   0.38 &     0.42 \\
Ceosalaries &  500 &  1 & 6 &  10.64 &     2.5 &       0.7 &   0.48 &     0.62 \\
    Housing &  506 & 13 & 2 &   2.34 &    1.76 &      0.99 &   0.67 &     0.75 \\
    Airfoil & 1503 &  5 & 4 &   2.77 &    1.21 &      0.86 &   0.59 &     0.79 \\
    Redwine & 1599 & 11 & 3 &   3.04 &    2.34 &      1.55 &   0.62 &      1.5 \\
    Abalone & 4177 &  7 & 3 &  14.94 &    9.38 &      5.57 &   2.77 &     6.36 \\
  Whitewine & 4898 & 11 & 3 &   3.98 &    3.05 &      1.86 &   1.03 &     2.55 \\
 Powerplant & 9568 &  4 & 5 &   2.71 &    1.76 &      0.53 &    0.3 &     1.56 \\
\bottomrule
\end{tabular}

